In [1]:
from IPython.display import HTML, display

#   How do we decide and learn in a volatile environment? 

## **Theoretical Framework**

## Limitations of current theoretical approaches 
#### Performing successfully in a volatile environment requires making fast, accurate decisions and updating those decisions given environmental feedback. However, accumulator models of choice-making, which model mechanisms internal to the decision, and reinforcement learning models, which involve how the outcome of those choices influence decision updates, are often isolated, despite their complementary goals. While drift diffusion modeling is capable of describing the speed and accuracy of choices, the decision parameters which govern those choices are assumed to remain static across trials, and while models of reinforcement learning explain how environmental feedback sculpts choice preferences, they fail to describe decision-making mechanisms. 

## Neural evidence for integration
#### Much research has shown that both action selection and reinforcement learning rely on the cortico-basal ganglia-thalamic (CBGT) circuit ([Bogacz & Larsen, 2011](https://www.mitpressjournals.org/doi/abs/10.1162/NECO_a_00103)), which contains three pathways with distinct effects on action selection: the direct (action facilitation), indirect (action suppression), and hyperdirect (immediate action cancellation) pathways. 
#### In this circuit, each component of the basal ganglia can be either facilitated or inhibited by direct and indirect action channels, where the direct pathway subdues tonic inhibition of the thalamus by the globus pallidus internal segment (GPi) to encourage action execution, and the indirect pathway activates the globus pallidus external segment (GPe) and subthalamic nucleus (STN) to increase GPi output and prevent action output. These indirect and direct pathways are also modulated by dopamine released during reinforcement learning via projections from the substantia nigra compacta (SNC). 

<tr><td><img src='cbgt_dunovanVerstynen.png' style='width: 700px;'>
*figure from [Dunovan and Verstynen 2018](https://www.biorxiv.org/content/early/2017/10/28/153676)*
    
#### Hyperdirect activation of the STN has been correlated with value-conflict ([Frank et al., 2015](http://www.jneurosci.org/content/35/2/485)), which is thought to affect evidence accumulation within the decision making framework. Additionally, there is evidence that the volatility of environmental feedback affects tonic dopamine levels in the striatum to regulate the explore-exploit tradeoff for potential actions ([Humphris, Khamassi, & Gurney, 2012](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3272648/)). Altogether, this evidence suggests that 1) reinforcement learning and decision making processes act as integrated parts of a whole, and 2) it points to the importance of value-conflict and volatility as key learning signals for the integration of decision making and reinforcement learning. 

#### I plan to explore how value conflict between competing actions (the degree to which the value associated with each action is similar) and the volatility of feedback (the change point frequency of mean value-action associations) influence adaptive decision making using a  drift-diffusion reinforcement learning model. 

## **Task**

#### In our task, we operationally define conflict as the difference in the probability of reward between actions and volatility as the rate of change in that difference. We plan to manipulate factors of volatility and conflict using a within-subjects design to form four conditions, with each participant performing **XX** trials per condition and two conditions per day until all conditions are complete.     

#### Reaction time and choice accuracy data will be collected using a two-alternative forced-choice task written in PsychoPy. On each trial, participants will be asked to choose the highest-reward target within 1 s. Then they will be shown the reward earned on each trial based on their decision. To prevent prepotent selections, the position of the rewarding target will be randomized across trials, with the target identity as the rewarding feature (rather than target position).  The instruction screen and the structure of a sample trial are below: 
<tr><td><img src='inst_p1.png' style='width: 700px;'><img src='inst_p2.png' style='width: 700px;'>
</td><td><img src='task.png' style='width: 700px;'></td></tr>

#### Each participant will receive a separate reward schedule. 
#### Below is a sample of one high conflict reward schedule, with the probability of reward on the y-axis. The most rewarding target gives a reward 65% of the time and the identity of the most rewarding target shifts probabilistically, approximately every 20 trials. <br>

<tr><td><img src='high_conflict.png' style='width: 500px;'>
#### A low conflict reward schedule is shown below. The high-value target gives a reward 85% of the time. <br>

<tr><td><img src='low_conflict.png' style='width: 500px;'>
    
#### Below is a sample of a high volatility reward schedule. The identity of the most rewarding target shifts approximately every 10 trials. <br>

<tr><td><img src='high_volatility.png' style='width: 500px;'>
#### A low volatility reward schedule is shown below. The identity of the most rewarding target shifts approximately every 30 trials. <br>

<tr><td><img src='low_volatility.png' style='width: 500px;'>



## **Hypotheses** 
<br>
#### *Conflict*

#### The rate of evidence accumulation [$v$] affects the speed of a decision, and so may vary with conflict, such that lower conflict conditions show greater drift rates and higher conflict conditions show slower drift rates. 
<br>

#### The degree to which evidence accumulation is biased toward the higher value target may also differ according to conflict. The starting point for evidence accumulation [$z$] may vary with conflict, with larger differences in prevous action values biasing the starting point for evidence accumulation toward the target of higher value, and smaller differences in value decreasing starting point bias (so that $z$ is closer to $a/2$). 

<br>
#### *Volatility*  
#### The decision threshold [$a$], which represents a global degree of caution regarding either action and regulates choice accuracy, may increase and decrease with change point probability [$\Omega$]. 
#### When the decision threshold increases with change point probability, the learner has more time to make a decision, so she may be shifted toward a more explorative learning policy. 

#### Increased volatility will increase learning rates [$\beta$]. 

#### Decision threshold and drift rate adaptation will likely combine to drive behavior, with the threshold shift driven by change point probability and affecting both targets, or by belief in the reward difference changing the starting point, biasing the decision toward the high value target. The drift rate change will be driven by belief in the reward difference.


## **Cognitive models**

#### Nomenclature

<center>**Learning signals (estimates from ideal observer)**</center>
$$
\begin{align}
{B_\Delta} = \textrm{signed belief in the reward difference between targets} && \Omega = \textrm{change point probability}\\
\end{align}
$$ 
<center>**Learning targets (decision parameters)** </center>
$$
\begin{align}
a = \textrm{boundary} && v = \textrm{execution drift rate}\\  
\end{align}
$$

<center>**Other parameters**</center> 
$$
\begin{align}
\sigma^2_n = \textrm{variance of the generative distribution} && \sigma^2_t = \textrm{estimated variance}\\ 
\phi = \textrm{model confidence} && H = \textrm{hazard rate}\\ 
r_{t} = \textrm{reward observed} && \alpha = \textrm{bayesian belief learning rate}\\
\delta = \textrm{reward prediction error}\\
\end{align}
$$
<br>

## Approach
We update the targets of learning, decision boundary height, the rate of evidence accumulation, and the starting point, using estimates of the reward difference between targets and the reward changepoint probability as learning signals. 

## Belief calculation
Given that $c$ = the chosen target and $u$ = the unchosen target, the belief in the mean of the distribution of reward differences on the next trial is calculated as:
$$B_{t+1, c} = B_{t, c}  + \alpha_t\delta_t$$

The unchosen target value decays to the pooled expected value of both targets, $E(r)$:
$$B_{t+1, u} = B_{t, u}(1-\Omega_t)+\Omega_tE(r)$$
<br>
$$E(r) = \frac{\bar{r}_{t_0} + \bar{r}_{t_1}}{2}$$

The signed belief in the reward difference between targets is calculated as the difference in belief for targets 0 and 1:
$$B_{\Delta_{t+1}} = B_{t,1} - B_{t,0}$$


Where the learning rate of the model [$\alpha$] is influenced by the change point probability [$\Omega$] and the model confidence [$\phi$]. The learning rate should be high if either 1) a change in the mean of the distribution of reward is likely [$\Omega$ is high] or 2) the estimate of the mean is highly imprecise [$\sigma^2_n$ is high]:
$$\alpha_t = \Omega_t + (1-\Omega)(1-\phi_t)$$
The prediction error, $\delta$, is the difference between the model belief and the reward difference observed:
$$\delta_t = r_t - B_t$$
Estimated variance is calculated as:
$$\sigma^2_t = \sigma^2_n + \frac{(1-\phi_t)\sigma^2_n}{\phi_t}$$
## Changepoint probability calculation
The changepoint probability is the likelihood that a new sample is drawn from the same Gaussian distribution centered about the current belief estimate of the model relative to the likelihood that a new sample is drawn from a uniform distribution. The changepoint probability will be close to 1 as the relative probability of a sample coming from a uniform distribution increases.
$$\Omega_t = \frac{U(r_{\Delta_{t}})H}{U(r_{\Delta_{t}})H + N(r_{\Delta_{t}}| B_{\Delta_{t}},\sigma^2_t)(1-H)}$$
The hazard rate is the global probability that the mean of the distribution has changed (calculated as the sum of change points over the total number of trials).
$$H = \frac{sum(cp_{trials})}{n_{trials}}$$



The model confidence [$\phi$] is a function of the changepoint probability [$\Omega$] and the variance of the generative distribution [$\sigma^2_n$]. The first term is the variance when a changepoint is assumed to have occurred. The second term is the variance conditional on no changepoint (slowly decaying uncertainty). The third term is the rise in uncertainty when the model is unsure whether a changepoint has occurred. The same terms are in the denominator with an added variance term to reflect uncertainty arising from noise.
$$RU_t = \frac{\Omega_t\sigma^2_n + (1-\Omega_t)(1-\phi_t)\sigma^2_n + \Omega_t(1-\Omega_t)(\delta_t\phi_t)^2}{\Omega_t\sigma^2_n + (1-\Omega_t)(1-\phi_t)\sigma^2_n + \Omega_t(1-\Omega_t)(\delta_t\phi_t)^2+\sigma^2_n}$$

$$\phi_{t+1} =  1 - RU$$

## Models

We propose three component cognitive models.<br>

**Adaptive drift rate model.** The drift rate alone may vary as a function of the estimated reward difference between targets, where an increased drift rate would speed evidence accumulation, decreasing reaction time and increasing choice accuracy:
$$v_{t+1} = \hat\beta{\cdot B_{\Delta_t}} + v_t$$


**Adaptive decision boundary model.** The boundary alone may adapt as a function of the change point probability, which would increase the window of evidence accumulation, increasing reaction time and choice accuracy:
$$a_{t+1} = \hat\beta\cdot\Omega_t + a_0$$


**Adaptive starting point model.** The starting point may adapt as a function of the estimated reward difference between targets, decreasing reaction time and increasing accuracy:
$$z_{t+1} = \hat\beta \cdot B_{\Delta_t} + z_0$$

As mentioned in the Hypotheses section, we hypothesize that a combination of decision threshold adaptation (whether value-driven, affecting the starting point for evidence accumulation, or volatility-driven, affecting the global caution associated with both actions) and value-driven drift rate adaptation will modulate behavior.

## **Simulations** 

## Component Decision Processes

## conflict  
Higher conflict conditions have lower drift rates, on average. The decision threshold also increases with the conflict, on average, though low conflict conditions show an immediate spike after the changepoint. 

<table><tr>
<td> <img src="drift_conflict_collapsed.png"  style="width: 900px;"/> </td>
<td> <img src="bound_conflict_collapsed.png"  style="width: 900px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="conflict_v.png"  style="width: 900px;"/> </td>
<td> <img src="conflict_a.png"  style="width: 900px;"/> </td>
</tr></table>

## volatility 
Higher volatility conditions have higher decision thresholds, on average. The drift rate  also decreases with volatility.

<table><tr>
<td> <img src="bound_volatility_collapsed.png"  style="width: 900px;"/> </td>
<td> <img src="drift_volatility_collapsed.png"  style="width: 900px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="vol_v.png"  style="width: 900px;"/> </td>
<td> <img src="vol_a.png"  style="width: 900px;"/> </td>
</tr></table>


## Simulated Behavior 

High conflict conditions reduce accuracy and reduce the amplitude of increases in reaction time following a changepoint. High volatility conditions increase accuracy more quickly than conditions of low volatility, but have lower asymptotic accuracy. High volatility conditions have a greater baseline reaction time than lower volatility conditions. 

Below, reaction times are phase-shifted for the high volatility conditions, with high volatility conditions exhibiting greater reaction times than low volatility conditions. Accuracy increases more quickly under conditions of high volatility but asymptotes at a lower value than for low volatility conditions. 

<table><tr>
<td> <img src="vol_rt.png"  style="width: 900px;"/> </td>
<td> <img src="vol_acc.png"  style="width: 900px;"/> </td>
</tr></table>

Below, reaction times increase more quickly under conditions of low conflict than high conflict, with the high conflict condition showing an extremely slow increase in reaction time and accuracy as the learner disambiguates the value difference between targets.

<table><tr>
<td> <img src="conflict_rt.png"  style="width: 900px;"/> </td>
<td> <img src="conflict_acc.png"  style="width: 900px;"/> </td>
</tr></table>

## Power

The true boundary is well-estimated across conflict and volatility conditions. Estimating boundary and drift rate separately has little impact on the final error in the boundary estimates. Estimates stabilize around 400 trials. 

<table><tr>
<td> <img src="vol_bound_power.png"  style="width: 900px;"/> </td>
<td> <img src="conflict_bound_power.png"  style="width: 900px;"/> </td>
</tr></table>

The true drift rate is somewhat well-estimated in low conflict and low volatility conditions. Single parameter estimates are slightly more accurate in these conditions. 

However, the drift rate estimates for high conflict and volatility conditions maintains a large steady state error of ~0.6-0.7. In these conditions, the single parameter estimates are no more accurate. 

<table><tr>
<td> <img src="vol_drift_power.png"  style="width: 900px;"/> </td>
<td> <img src="conflict_drift_power.png"  style="width: 900px;"/> </td>
</tr></table>

## DIC Scores 

The combined drift rate and threshold adaptation model loses the least information, as measured by Deviance Information Criterion scores. The separation between the bound and combined models increases with the number of trials. They begin to be clearly separated given 600 trials. 

<img src="DIC_lines.png"  style="width: 700px;"/> 


## **Model Comparison & fitting procedures**

### **Parameter estimates.** Parameter estimates will be derived from behavioral data using hierarchical Bayesian estimation. This procedure was chosen for its ability to recover models for smaller datasets, its robustness to outliers and multiple comparisons, and distribution-level expression of estimation uncertainty (as opposed to using parametric  estimates of confidence around a point estimate). 

### **Model comparison & falsification.** To quantify the likelihood of observing the experimental data given each model, we will compare model deviance information criterion (DIC) scores. In addition, we will simulate the data using the best-fitting parameter values to check 1) whether the same reaction time and accuracy profiles are detectable in the empirical data (the model's generative performance), and 2) whether there is a statistical difference between observed and simulated data (see [Palminteri et al., 2017](https://www.ncbi.nlm.nih.gov/pubmed/28476348) for instances in which relative information criterion scores can be misleading without absolute falsification criteria). 


## **Data storage and availability**

### All  data, experiment/analysis code, and *a priori* hypotheses will be stored using a project [repository](https://github.com/kmbond/volatileValues) on GitHub, and updated throughout the project timeline. Ultimately, this repository will be linked to a public project using the Open Science Framework.